<center> <h1> &#127756 Parcial I Física Computacional &#127826 </h1> </center>

<center> <h3> <em> Sebastián Marín Ruiz, Gabriela E. Díaz Mejía </em> </h3> </center>

<center> <h4> <em> Programa de Física, Facultad de Ciencias Básicas y Tecnologías </em> </h4> </center>
<br><br>
<center> Trabajo realizado en: </center>

![logo jupyter](jup_logo.png)

In [1]:
# Librerías
%autosave 60
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

Autosaving every 60 seconds


> **Nota:** En este trabajo, se eligieron las primeras 31 columnas (que se exponen en las siguiente celda) para facilitar el análisis de nuestra base de datos. Además, consideramos que las variables elegidas son las más relevantes para nuestro estudio.

In [2]:
# Abriendo archivo csv
datos = pd.read_csv("METABRIC_RNA_Mutation.csv")
datos.columns[:31]

C:\Users\s\AppData\Local\Temp\ipykernel_17344\565455140.py:2: DtypeWarning: Columns (678,688,690,692) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("METABRIC_RNA_Mutation.csv")


Index(['patient_id', 'age_at_diagnosis', 'type_of_breast_surgery',
       'cancer_type', 'cancer_type_detailed', 'cellularity', 'chemotherapy',
       'pam50_+_claudin-low_subtype', 'cohort', 'er_status_measured_by_ihc',
       'er_status', 'neoplasm_histologic_grade',
       'her2_status_measured_by_snp6', 'her2_status',
       'tumor_other_histologic_subtype', 'hormone_therapy',
       'inferred_menopausal_state', 'integrative_cluster',
       'primary_tumor_laterality', 'lymph_nodes_examined_positive',
       'mutation_count', 'nottingham_prognostic_index', 'oncotree_code',
       'overall_survival_months', 'overall_survival', 'pr_status',
       'radio_therapy', '3-gene_classifier_subtype', 'tumor_size',
       'tumor_stage', 'death_from_cancer'],
      dtype='object')

> <h2> Primer paso: selección y explicación de variables </h2>
>
> Inicialmente, se seleccionan las variables que se utilizarán y se analiza cada columna individualmente para determinar si tiene un significado coherente en el modelo que se vaya a realizar. Así, la contribución de cada columna es la siguiente:
>
> - **patient_id:** Es la identificación única de cada paciente
> - **type_of_breast_surgery:** Expone el tipo de cirugía a la que fue sometida cada paciente.
> - **cancer_type_detailed:** Contiene la información específica del tipo de cáncer detectado.
> - **cellularity:**
> - **neoplasm_histologic_grade:**
> - **lymph_nodes_examined_positive:**
> - **nottingham_prognostic_index:**
> - **overall_survival_months:**
> - **tumor_stage:**
> - **death_from_cancer:**

In [3]:
# Tabla de las variables seleccionadas
analisis = pd.DataFrame(datos, columns=["patient_id","type_of_breast_surgery", 
                                      "cancer_type_detailed", "cellularity","neoplasm_histologic_grade",
                                      "lymph_nodes_examined_positive", "nottingham_prognostic_index",
                                      "overall_survival_months","tumor_stage", "death_from_cancer" ])
analisis.head()

,patient_id,type_of_breast_surgery,cancer_type_detailed,cellularity,neoplasm_histologic_grade,lymph_nodes_examined_positive,nottingham_prognostic_index,overall_survival_months,tumor_stage,death_from_cancer
0,0,MASTECTOMY,Breast Invasive Ductal Carcinoma,NaN,3.0,10.0,6.044,140.500000,2.0,Living
1,2,BREAST CONSERVING,Breast Invasive Ductal Carcinoma,High,3.0,0.0,4.020,84.633333,1.0,Living
2,5,MASTECTOMY,Breast Invasive Ductal Carcinoma,High,2.0,1.0,4.030,163.700000,2.0,Died of Disease
3,6,MASTECTOMY,Breast Mixed Ductal and Lobular Carcinoma,Moderate,2.0,3.0,4.050,164.933333,2.0,Living
4,8,MASTECTOMY,Breast Mixed Ductal and Lobular Carcinoma,High,3.0,8.0,6.080,41.366667,2.0,Died of Disease


> <h2> Segundo paso </h2>
>
> Ahora, se deben eliminar aquellos datos que no aporten a la base de datos, es decir, los valores nulos; ya que pueden generar errores en los diferentes parámetros estadísticos que se calculen como la mediana, el promedio, la media, entre otros.
>
> El propósito de la siguiente celda es eliminar aquellas filas cuyos valores **NO** nulos ocupen al menos un 80% de los datos, ya que consideramos que si faltan al menos dos datos de cada paciente, puede dificultar el diagnóstico del mismo. El resultado fue que se eliminaron 6 filas. Posteriormente, se rectificó que no hubiesen identificaciones duplicadas en la base de datos, ya que es el único dato irrepetible.

In [4]:
# Porcentaje de valores no nulos para la limpieza
porcentaje = 0.8

# Eliminación de valores nulos con cierto umbral
limpieza_1 = analisis.dropna(thresh = porcentaje*len(analisis.columns))
limpieza_1 = limpieza_1.drop_duplicates(["patient_id"])
limpieza_1

# Nota: Prestar atención a cualquier columna que:
# Esté mal formateada,
# Requiere más datos o mucho procesamiento para convertirse en una característica útil
# Contiene información redundante

,patient_id,type_of_breast_surgery,cancer_type_detailed,cellularity,neoplasm_histologic_grade,lymph_nodes_examined_positive,nottingham_prognostic_index,overall_survival_months,tumor_stage,death_from_cancer
0,0,MASTECTOMY,Breast Invasive Ductal Carcinoma,NaN,3.0,10.0,6.044,140.500000,2.0,Living
1,2,BREAST CONSERVING,Breast Invasive Ductal Carcinoma,High,3.0,0.0,4.020,84.633333,1.0,Living
2,5,MASTECTOMY,Breast Invasive Ductal Carcinoma,High,2.0,1.0,4.030,163.700000,2.0,Died of Disease
3,6,MASTECTOMY,Breast Mixed Ductal and Lobular Carcinoma,Moderate,2.0,3.0,4.050,164.933333,2.0,Living
4,8,MASTECTOMY,Breast Mixed Ductal and Lobular Carcinoma,High,3.0,8.0,6.080,41.366667,2.0,Died of Disease
...,...,...,...,...,...,...,...,...,...,...
1899,7295,BREAST CONSERVING,Breast Invasive Lobular Carcinoma,High,3.0,1.0,5.050,196.866667,NaN,Living
1900,7296,MASTECTOMY,Breast Invasive Ductal Carcinoma,High,3.0,1.0,5.040,44.733333,NaN,Died of Disease
1901,7297,MASTECTOMY,Breast Invasive Ductal Carcinoma,High,3.0,45.0,6.050,175.966667,NaN,Died of Disease
1902,7298,MASTECTOMY,Breast Invasive Ductal Carcinoma,Moderate,2.0,12.0,5.050,86.233333,NaN,Died of Other Causes


> <h2> Propuesta de ecuación </h2>
>
> En nuestro análisis, decidimos predecir... **Propongo el estado del tumor (tumor_stage) o la celularidad (cellularity)**

In [6]:
# Planteamiento de ecuación
# Creación de matriz de variables independientes
xi = limpieza_1.drop(["tumor_stage"], axis = 1)

# Creación de vector de variable dependiente
y = limpieza_1.loc[:, "tumor_stage"]

# Rango de la variable dependiente
#print(y.max())
#print(y.min())

# Contador de valores nulos en cada columna
#for i in limpieza_1.columns:
    #y = limpieza_1.loc[:, i]
    #print(y.isnull().sum())
    
# Modelo de regresión lineal
MRL = smf.ols("tumor_stage ~ patient_id + type_of_breast_surgery + cancer_type_detailed +\
              cellularity + neoplasm_histologic_grade + lymph_nodes_examined_positive +\
              nottingham_prognostic_index + overall_survival_months + death_from_cancer", 
              limpieza_1).fit()
print(MRL.summary())
print(MRL.params)

#sns.scatterplot(data=a,x=a["age_at_diagnosis"], y=a
                #["cellularity"])

0
17
14
52
67
0
0
0
496
1


> <h1> Cosas por hacer </h1>
>
> <h2> Limpieza de datos </h2>
>
> - Elegir variable dependiente
> - Explicar cada variable independiente
> - Proponer la ecuación lineal
> - Redefinir las variables categóricas
> - Normalizar la base de datos
> - Calcular parámetros estadísticos
>
> <h2> Prueba de hipótesis </h2>
>
> - Plantear hipótesis nula y alternativa
> - Graficar distribución
> - Plantear un modelo que se ajuste a la distribución
> - Aceptar o rechazar hipótesis nula
>
> <h2> Prueba ANOVA </h2>
>
> -